In [1]:
import cadquery as cq

# 3D 객체 생성
# 단위: mm (10cm = 100mm, 15cm = 150mm, 8cm = 80mm, 5cm 지름 = 25mm 반지름)
result = (
    cq.Workplane("XY")
    .box(100, 150, 80)           # Width 10cm, Depth 15cm, Height 8cm
    .faces(">Z")                 # 윗면 선택
    .workplane()                 # 윗면에 작업 평면 설정
    .circle(25)                  # 지름 5cm (반지름 25mm) 원
    .cutThruAll()                # 아래까지 관통 구멍
)

# 1. STEP 파일로 저장 (3D 표준 형식 - 추천!)
cq.exporters.export(result, 'test_part.step')
print("✅ STEP 파일 저장 완료: test_part.step")

# 2. STL 파일로 저장 (3D 프린팅/메시 형식)
cq.exporters.export(result, 'test_part.stl')
print("✅ STL 파일 저장 완료: test_part.stl")

# 3. DXF는 2D 형식이므로 3가지 투영도 생성
# 정면도 (Width x Height)
front_view = (
    cq.Workplane("XZ")
    .rect(100, 80)
    .extrude(1)
)
# DXF 직접 내보내기는 제한적이므로 SVG로 대체하거나
# 수동으로 DXF 생성 필요

# 대안: ezdxf로 직접 2D 투영 그리기
import ezdxf

# 정면도 DXF (Width x Height)
doc_front = ezdxf.new('R2010')
msp_front = doc_front.modelspace()
# 외곽선
msp_front.add_lwpolyline([(-50, -40), (50, -40), (50, 40), (-50, 40), (-50, -40)])
# 구멍 표시 (정면에서는 원으로 보임)
msp_front.add_circle((0, 0), 25)
doc_front.saveas('test_part_front.dxf')
print("✅ 정면도 DXF 저장 완료: test_part_front.dxf")

# 측면도 DXF (Depth x Height)
doc_side = ezdxf.new('R2010')
msp_side = doc_side.modelspace()
# 외곽선
msp_side.add_lwpolyline([(-75, -40), (75, -40), (75, 40), (-75, 40), (-75, -40)])
# 구멍 표시 (측면에서는 원으로 보임)
msp_side.add_circle((0, 0), 25)
doc_side.saveas('test_part_side.dxf')
print("✅ 측면도 DXF 저장 완료: test_part_side.dxf")

# 평면도 DXF (Width x Depth) - 구멍이 보이는 뷰
doc_top = ezdxf.new('R2010')
msp_top = doc_top.modelspace()
# 외곽선
msp_top.add_lwpolyline([(-50, -75), (50, -75), (50, 75), (-50, 75), (-50, -75)])
# 구멍 (원)
msp_top.add_circle((0, 0), 25)
doc_top.saveas('test_part_top.dxf')
print("✅ 평면도 DXF 저장 완료: test_part_top.dxf")

print("\n" + "="*50)
print("📁 생성된 파일:")
print("  1. test_part.step      - 3D 모델 (CAD 표준)")
print("  2. test_part.stl       - 3D 모델 (메시)")
print("  3. test_part_front.dxf - 2D 정면도 (Width x Height)")
print("  4. test_part_side.dxf  - 2D 측면도 (Depth x Height)")
print("  5. test_part_top.dxf   - 2D 평면도 (Width x Depth)")
print("="*50)

# 추가: 시각화 (선택사항)
try:
    from jupyter_cadquery.viewer.client import show
    show(result)
    print("\n🎨 3D 모델 시각화 표시됨")
except:
    print("\n💡 Jupyter 환경에서 show(result)로 3D 모델을 볼 수 있습니다")

✅ STEP 파일 저장 완료: test_part.step
✅ STL 파일 저장 완료: test_part.stl
✅ 정면도 DXF 저장 완료: test_part_front.dxf
✅ 측면도 DXF 저장 완료: test_part_side.dxf
✅ 평면도 DXF 저장 완료: test_part_top.dxf

📁 생성된 파일:
  1. test_part.step      - 3D 모델 (CAD 표준)
  2. test_part.stl       - 3D 모델 (메시)
  3. test_part_front.dxf - 2D 정면도 (Width x Height)
  4. test_part_side.dxf  - 2D 측면도 (Depth x Height)
  5. test_part_top.dxf   - 2D 평면도 (Width x Depth)

💡 Jupyter 환경에서 show(result)로 3D 모델을 볼 수 있습니다


In [2]:
import ezdxf

# DXF 파일 열기
doc = ezdxf.readfile('test_part_top.dxf')
msp = doc.modelspace()

# 치수 추가
msp.add_linear_dim(
    base=(0, -80),
    p1=(-50, -75),
    p2=(50, -75)
)

# 저장
doc.saveas('test_part_top_dimensioned.dxf')

In [3]:
import ezdxf
from ezdxf.enums import TextEntityAlignment

# 1. 기본 DXF 파일 열기 (또는 새로 생성)
doc = ezdxf.readfile('test_part_top.dxf')
msp = doc.modelspace()

# 2. 텍스트 스타일 설정 (중요!)
if 'Standard' not in doc.styles:
    doc.styles.new('Standard', dxfattribs={'font': 'arial.ttf'})

# 3. 치수 스타일 생성 및 상세 설정
dimstyle = doc.dimstyles.new('CUSTOM_DIM')

# 치수 텍스트 설정
dimstyle.dxf.dimtxt = 5.0          # 텍스트 높이
dimstyle.dxf.dimasz = 3.0          # 화살표 크기
dimstyle.dxf.dimexe = 2.0          # 치수 보조선 연장
dimstyle.dxf.dimexo = 1.0          # 치수 보조선 오프셋
dimstyle.dxf.dimtad = 1            # 텍스트 위치 (1 = 치수선 위)
dimstyle.dxf.dimgap = 1.0          # 텍스트와 치수선 간격

# 텍스트 스타일 지정
dimstyle.dxf.dimtxsty = 'Standard'

# 단위 설정
dimstyle.dxf.dimdec = 0            # 소수점 자릿수
dimstyle.dxf.dimdsep = ord('.')    # 소수점 구분자

# 4. 수평 치수 추가 (Width: 100mm)
dim1 = msp.add_linear_dim(
    base=(0, -85, 0),              # 치수선 위치
    p1=(-50, -75, 0),              # 시작점
    p2=(50, -75, 0),               # 끝점
    dimstyle='CUSTOM_DIM',
    override={
        'dimtxt': 5.0,             # 텍스트 크기 강제 지정
    }
)
dim1.render()  # 렌더링 - 중요!

# 5. 수직 치수 추가 (Depth: 150mm)
dim2 = msp.add_linear_dim(
    base=(-60, 0, 0),
    p1=(-50, -75, 0),
    p2=(-50, 75, 0),
    angle=90,                       # 수직 치수
    dimstyle='CUSTOM_DIM',
    override={
        'dimtxt': 5.0,
    }
)
dim2.render()

# 6. 구멍 지름 치수 추가 (Diameter: 50mm)
dim3 = msp.add_diameter_dim(
    center=(0, 0, 0),               # 원 중심
    radius=25,                      # 반지름
    angle=45,                       # 치수선 각도
    dimstyle='CUSTOM_DIM',
    override={
        'dimtxt': 5.0,
    }
)
dim3.render()

# 7. 추가 주석 텍스트 (확실하게)
# 폭 표기
text1 = msp.add_text(
    "100",
    height=6,
    dxfattribs={
        'style': 'Standard',
        'color': 1  # 빨간색
    }
)
text1.set_placement((0, -90), align=TextEntityAlignment.MIDDLE_CENTER)

# 깊이 표기
text2 = msp.add_text(
    "150",
    height=6,
    rotation=90,
    dxfattribs={
        'style': 'Standard',
        'color': 1
    }
)
text2.set_placement((-70, 0), align=TextEntityAlignment.MIDDLE_CENTER)

# 지름 표기
text3 = msp.add_text(
    "Ø50",
    height=6,
    dxfattribs={
        'style': 'Standard',
        'color': 1
    }
)
text3.set_placement((20, 20), align=TextEntityAlignment.LEFT)

# 8. 제목 및 설명 추가
title = msp.add_text(
    "평면도 (TOP VIEW)",
    height=8,
    dxfattribs={
        'style': 'Standard',
        'color': 2  # 노란색
    }
)
title.set_placement((0, 95), align=TextEntityAlignment.MIDDLE_CENTER)

# 단위 표기
unit_text = msp.add_text(
    "단위: mm",
    height=4,
    dxfattribs={
        'style': 'Standard',
        'color': 7  # 흰색/검정
    }
)
unit_text.set_placement((60, 90), align=TextEntityAlignment.LEFT)

# 9. 저장
doc.saveas('test_part_top_dimensioned_fixed.dxf')
print("✅ 치수 값이 포함된 DXF 파일 저장 완료!")
print("📁 파일명: test_part_top_dimensioned_fixed.dxf")
print("\n표기된 치수:")
print("  - 폭(Width): 100mm")
print("  - 깊이(Depth): 150mm")
print("  - 구멍 지름: Ø50mm")

✅ 치수 값이 포함된 DXF 파일 저장 완료!
📁 파일명: test_part_top_dimensioned_fixed.dxf

표기된 치수:
  - 폭(Width): 100mm
  - 깊이(Depth): 150mm
  - 구멍 지름: Ø50mm


In [1]:
import ezdxf

def detailed_diagnosis(filename):
    doc = ezdxf.readfile(filename)
    msp = doc.modelspace()
    
    print("\n=== 모든 엔티티 타입 ===")
    entity_types = {}
    for entity in msp:
        etype = entity.dxftype()
        entity_types[etype] = entity_types.get(etype, 0) + 1
    
    for etype, count in sorted(entity_types.items()):
        print(f"{etype}: {count}개")
    
    print("\n=== 블록 내부 확인 ===")
    for block in doc.blocks:
        if not block.name.startswith('*'):
            entities_in_block = list(block)
            if entities_in_block:
                print(f"\n블록명: {block.name} ({len(entities_in_block)}개 엔티티)")
                block_types = {}
                for entity in entities_in_block:
                    etype = entity.dxftype()
                    block_types[etype] = block_types.get(etype, 0) + 1
                for etype, count in sorted(block_types.items()):
                    print(f"  {etype}: {count}개")
                
                # 블록 내부의 TEXT 확인
                for entity in block:
                    if entity.dxftype() == 'TEXT':
                        print(f"    TEXT 발견: '{entity.dxf.text}'")
    
    print("\n=== INSERT 엔티티 확인 ===")
    inserts = list(msp.query('INSERT'))
    print(f"총 INSERT: {len(inserts)}개")
    for insert in inserts[:5]:  # 처음 5개만
        print(f"  블록명: {insert.dxf.name}, 위치: ({insert.dxf.insert.x:.1f}, {insert.dxf.insert.y:.1f})")

# 실행
detailed_diagnosis('data\\gear-disk\\Gear Disk dxf File.dxf')


=== 모든 엔티티 타입 ===
ARC: 50개
CIRCLE: 6개
LEADER: 1개
LINE: 1007개
TEXT: 14개

=== 블록 내부 확인 ===

=== INSERT 엔티티 확인 ===
총 INSERT: 0개


In [2]:
import ezdxf

def check_small_lines(filename):
    doc = ezdxf.readfile(filename)
    msp = doc.modelspace()
    
    print("=== 짧은 LINE 분석 (5mm 이하) ===")
    short_lines = []
    
    for line in msp.query('LINE'):
        start = line.dxf.start
        end = line.dxf.end
        length = ((end[0] - start[0])**2 + (end[1] - start[1])**2)**0.5
        
        if length < 5:
            short_lines.append({
                'length': length,
                'layer': line.dxf.layer,
                'start': (start.x, start.y),
                'end': (end.x, end.y)
            })
    
    print(f"5mm 이하 짧은 LINE: {len(short_lines)}개")
    
    # 레이어별 분류
    layer_count = {}
    for line in short_lines:
        layer = line['layer']
        layer_count[layer] = layer_count.get(layer, 0) + 1
    
    print("\n레이어별 분포:")
    for layer, count in sorted(layer_count.items(), key=lambda x: x[1], reverse=True):
        print(f"  {layer}: {count}개")
    
    # 샘플 출력
    print(f"\n샘플 (처음 10개):")
    for i, line in enumerate(short_lines[:10]):
        print(f"  {i+1}. 길이: {line['length']:.2f}mm, 레이어: {line['layer']}")

check_small_lines('data\\gear-disk\\Gear Disk dxf File.dxf')

=== 짧은 LINE 분석 (5mm 이하) ===
5mm 이하 짧은 LINE: 1007개

레이어별 분포:
  Unnamed Level 1: 1007개

샘플 (처음 10개):
  1. 길이: 0.90mm, 레이어: Unnamed Level 1
  2. 길이: 0.10mm, 레이어: Unnamed Level 1
  3. 길이: 0.26mm, 레이어: Unnamed Level 1
  4. 길이: 0.44mm, 레이어: Unnamed Level 1
  5. 길이: 0.04mm, 레이어: Unnamed Level 1
  6. 길이: 0.10mm, 레이어: Unnamed Level 1
  7. 길이: 0.26mm, 레이어: Unnamed Level 1
  8. 길이: 0.44mm, 레이어: Unnamed Level 1
  9. 길이: 0.90mm, 레이어: Unnamed Level 1
  10. 길이: 0.04mm, 레이어: Unnamed Level 1
